In [12]:
import pandas as pd
import random
import numpy as np
import pickle
import joblib # Часто лучше для объектов scikit-learn
import json
import re # Для обработки времени выполнения
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce # Для TargetEncoder

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import sklearn
import xgboost
print(f"scikit-learn version: {sklearn.__version__}")
print(f"xgboost version: {xgboost.__version__}")


# Настройки для воспроизводимости
random.seed(42)
np.random.seed(42)

# Настройки отображения pandas и matplotlib
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")

scikit-learn version: 1.6.1
xgboost version: 3.0.1


In [13]:
# Загрузка данных
try:
    data = pd.read_csv("../data/all_data.csv")
except FileNotFoundError:
    print("Файл ../data/all_data.csv не найден. Проверьте путь к файлу.")
    # Здесь можно либо остановить выполнение, либо попытаться загрузить из другого места
    # Для примера, если файл не найден, создадим заглушку DataFrame, чтобы код не падал дальше.
    # В реальном проекте здесь должно быть корректное завершение или обработка ошибки.
    data = pd.DataFrame() # Заглушка

if not data.empty:
    print("Данные успешно загружены.")
    print("Размер исходных данных:", data.shape)
    print("Первые 5 строк:")
    print(data.head())

    # Анализ пропусков
    nan_data_stats = (data.isnull().mean() * 100).reset_index()
    nan_data_stats.columns = ["column_name", "percentage"]
    nan_data_stats.sort_values("percentage", ascending=False, inplace=True)
    print("\nПроцент пропусков в столбцах (топ 24):")
    print(nan_data_stats.head(24))

    # Удаление строк, где целевая переменная 'worldwide' отсутствует
    data_initial_rows = len(data)
    data = data.dropna(subset=['worldwide'])
    print(f"\nУдалено {data_initial_rows - len(data)} строк из-за отсутствия значения в 'worldwide'.")
    print("Размер данных после удаления строк с NaN в 'worldwide':", data.shape)
else:
    print("Не удалось загрузить данные. Дальнейшее выполнение скрипта может быть некорректным.")

Данные успешно загружены.
Размер исходных данных: (5719, 24)
Первые 5 строк:
    movie_id     movie_title  movie_year            director          writer  \
0  tt0118589         Glitter        2001  Vondie Curtis-Hall  Cheryl L. West   
1  tt0120630     Chicken Run        2000          Peter Lord      Peter Lord   
2  tt0120667  Fantastic Four        2005           Tim Story      Mark Frost   
3  tt0120679           Frida        2002        Julie Taymor  Hayden Herrera   
4  tt0120681       From Hell        2001       Albert Hughes      Alan Moore   

             producer                composer   cinematographer  \
0       Laurence Mark       Terence Blanchard  Geoffrey Simpson   
1          Peter Lord  Harry Gregson-Williams      Simon Jacobs   
2            Avi Arad             John Ottman       Oliver Wood   
3  Lindsay Flickinger       Elliot Goldenthal    Rodrigo Prieto   
4        Jane Hamsher            Trevor Jones      Peter Deming   

    main_actor_1     main_actor_2   mai

In [14]:
if not data.empty:
    # Разделение на обучающую и тестовую выборки
    # Тестовая выборка будет сохранена в test.csv для будущего использования скриптом test.py
    train_data, test_data_for_file = train_test_split(data, test_size=0.3, random_state=42)

    # Сохранение тестовых данных в файл test.csv
    # Этот файл будет использоваться отдельным скриптом test.py для предсказаний
    test_data_for_file.to_csv("test.csv", index=False)
    print(f"\nТестовые данные сохранены в test.csv (размер: {test_data_for_file.shape})")
    print(f"Обучающие данные (размер: {train_data.shape})")

    # Сохраним оригинальное название фильма из обучающей выборки для возможного анализа
    train_movie_title_original = train_data['movie_title'].copy() # Сохраняем до удаления столбца

    # Определение столбцов для удаления (эти столбцы не будут использоваться для обучения)
    columns_to_drop_initial = ['movie_id', 'movie_title', 'link']
    train_data = train_data.drop(columns=columns_to_drop_initial, errors='ignore') # errors='ignore' если какой-то колонки уже нет
    print("\nСтолбцы, удаленные из train_data:", columns_to_drop_initial)
    print("Первые 5 строк train_data после удаления столбцов:")
    print(train_data.head())
else:
    print("Пропуск разделения данных, так как исходные данные не были загружены.")
    train_data = pd.DataFrame() # Заглушка


Тестовые данные сохранены в test.csv (размер: (1714, 24))
Обучающие данные (размер: (3998, 24))

Столбцы, удаленные из train_data: ['movie_id', 'movie_title', 'link']
Первые 5 строк train_data после удаления столбцов:
      movie_year             director              writer            producer  \
4482        2008          Sanaa Hamri  Elizabeth Chandler  Debra Martin Chase   
4803        2014          R.J. Cutler        Shauna Cross    Alison Greenspan   
4890        2016       Sharon Maguire      Helen Fielding           Tim Bevan   
5509        2016        Travis Knight         Marc Haimes       Travis Knight   
2124        2015  Alfonso Gomez-Rejon       Jesse Andrews       Jeremy Dawson   

              composer   cinematographer        main_actor_1   main_actor_2  \
4482    Rachel Portman       Jim Denault     America Ferrera  Alexis Bledel   
4803    Heitor Pereira    John de Borman  Chloë Grace Moretz  Mireille Enos   
4890   Craig Armstrong       Andrew Dunn     Renée Zellwe

In [15]:
# Инициализация словарей для сохранения карт преобразований
# Эти словари будут заполняться ПО ХОДУ предобработки данных
# и затем сохранены в JSON для использования в test.py

experience_maps_to_save = {}
grouped_imputation_maps_to_save = {}

In [16]:
if not train_data.empty:
    # 1. Создание признаков "опыта" для съемочной группы
    personnel_cols_for_experience = ["director", "writer", "producer", "composer", "cinematographer"]
    print("\nСоздание признаков опыта для съемочной группы...")
    for col in personnel_cols_for_experience:
        if col in train_data.columns:
            counts = train_data[col].value_counts()
            experience_maps_to_save[f"{col}_experience_map"] = counts.to_dict() # СОХРАНЯЕМ КАРТУ
            train_data[f"{col}_experience"] = train_data[col].map(counts)
            train_data[f"{col}_experience"].fillna(0, inplace=True) # Заполняем NaN (если значение не встретилось) нулем
            print(f"  Создан признак {col}_experience")
        else:
            print(f"  Предупреждение: столбец {col} не найден в train_data для создания опыта.")

    # 2. Создание признаков "опыта" для главных актеров и суммарной "популярности" состава
    actor_experience_cols = []
    print("\nСоздание признаков опыта для актеров...")
    for i in range(1, 5): # main_actor_1, main_actor_2, main_actor_3, main_actor_4
        col = f"main_actor_{i}"
        if col in train_data.columns:
            counts = train_data[col].value_counts()
            experience_maps_to_save[f"{col}_experience_map"] = counts.to_dict() # СОХРАНЯЕМ КАРТУ
            experience_col_name = f"{col}_experience"
            train_data[experience_col_name] = train_data[col].map(counts)
            train_data[experience_col_name].fillna(0, inplace=True)
            actor_experience_cols.append(experience_col_name)
            print(f"  Создан признак {experience_col_name}")
        else:
            print(f"  Предупреждение: столбец {col} не найден в train_data для создания опыта.")

    if actor_experience_cols:
        train_data["cast_popularity"] = train_data[actor_experience_cols].sum(axis=1)
        print("  Создан признак cast_popularity")
    else:
        train_data["cast_popularity"] = 0 # Если актеров нет, популярность 0
        print("  Признак cast_popularity установлен в 0 (актерские колонки не найдены/обработаны).")


    # 3. Преобразование 'run_time' в минуты
    def convert_runtime_to_minutes(value):
        match = re.match(r'(?:(\d+)\s*hr)?\s*(?:(\d+)\s*min)?', str(value))
        if match:
            hours = int(match.group(1)) if match.group(1) else 0
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
        return None # если формат не распознан или значение NaN

    if 'run_time' in train_data.columns:
        train_data['run_time'] = train_data['run_time'].apply(convert_runtime_to_minutes)
        print("\nСтолбец 'run_time' преобразован в минуты.")
    else:
        print("\nПредупреждение: столбец 'run_time' не найден в train_data.")

    print("\nПервые 5 строк train_data после создания признаков:")
    print(train_data.head())
else:
    print("Пропуск Feature Engineering, так как train_data пуст.")


Создание признаков опыта для съемочной группы...
  Создан признак director_experience
  Создан признак writer_experience
  Создан признак producer_experience
  Создан признак composer_experience
  Создан признак cinematographer_experience

Создание признаков опыта для актеров...
  Создан признак main_actor_1_experience
  Создан признак main_actor_2_experience
  Создан признак main_actor_3_experience
  Создан признак main_actor_4_experience
  Создан признак cast_popularity

Столбец 'run_time' преобразован в минуты.

Первые 5 строк train_data после создания признаков:
      movie_year             director              writer            producer  \
4482        2008          Sanaa Hamri  Elizabeth Chandler  Debra Martin Chase   
4803        2014          R.J. Cutler        Shauna Cross    Alison Greenspan   
4890        2016       Sharon Maguire      Helen Fielding           Tim Bevan   
5509        2016        Travis Knight         Marc Haimes       Travis Knight   
2124        2015  Alf

/var/folders/ls/btrdy60n12n_lk3plpt7wqlh0000gn/T/ipykernel_29645/3883809262.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[f"{col}_experience"].fillna(0, inplace=True) # Заполняем NaN (если значение не встретилось) нулем
/var/folders/ls/btrdy60n12n_lk3plpt7wqlh0000gn/T/ipykernel_29645/3883809262.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work becau

In [17]:
if not train_data.empty:
    # 1. Импутация числовых признаков медианой
    num_cols_for_imputation = train_data.select_dtypes(include=[np.number]).columns.tolist()
    # Исключаем целевую переменную 'worldwide', если она еще числовая и попала в список
    if 'worldwide' in num_cols_for_imputation:
        num_cols_for_imputation.remove('worldwide')
    
    if num_cols_for_imputation: # Если есть числовые колонки для импутации
        num_imputer = SimpleImputer(strategy='median')
        train_data[num_cols_for_imputation] = num_imputer.fit_transform(train_data[num_cols_for_imputation])
        print(f"\nЧисловые пропуски в {num_cols_for_imputation} заполнены медианой.")
    else:
        print("\nНет числовых колонок для импутации (кроме, возможно, 'worldwide').")
        num_imputer = None # Инициализируем на случай, если не создался

    # 2. Групповая импутация для некоторых категориальных признаков (модой по 'director')
    # Эти столбцы должны быть категориальными (object)
    cols_to_fill_by_director_mode = ['cinematographer', 'composer', 'producer', 'writer']
    print("\nГрупповая импутация для категориальных признаков (модой по 'director')...")
    if 'director' in train_data.columns:
        for col_to_impute in cols_to_fill_by_director_mode:
            if col_to_impute in train_data.columns:
                # Создаем карту: director -> mode_of_col_to_impute
                director_to_mode_map = train_data.groupby('director')[col_to_impute].apply(
                    lambda x: x.mode().iloc[0] if not x.mode().empty and not x.mode().isnull().all() else np.nan
                )
                # Сохраняем карту для артефактов
                grouped_imputation_maps_to_save[f"{col_to_impute}_director_mode_map"] = director_to_mode_map.to_dict()
                
                # Применяем карту для заполнения пропусков
                mapped_modes = train_data['director'].map(director_to_mode_map)
                train_data[col_to_impute] = train_data[col_to_impute].fillna(mapped_modes)
                print(f"  Пропуски в '{col_to_impute}' заполнены модой по группам 'director'.")
            else:
                print(f"  Предупреждение: столбец {col_to_impute} не найден для групповой импутации.")
    else:
        print("  Предупреждение: столбец 'director' не найден, групповая импутация не будет выполнена.")

    # 3. Заполнение специфических пропусков значением 'Unknown'
    cols_to_fill_unknown_specific = ['genre_2', 'genre_3', 'genre_4', 'main_actor_4']
    print("\nЗаполнение специфических пропусков значением 'Unknown'...")
    for col in cols_to_fill_unknown_specific:
        if col in train_data.columns:
            train_data[col] = train_data[col].fillna('Unknown')
            print(f"  Пропуски в '{col}' заполнены 'Unknown'.")
        else:
            print(f"  Предупреждение: столбец {col} не найден для заполнения 'Unknown'.")


    # 4. Импутация оставшихся категориальных признаков модой
    # Важно: cat_cols_for_imputation должен определяться ДО кодирования (OHE, Target Encoding)
    cat_cols_for_imputation = train_data.select_dtypes(include=['object']).columns.tolist()
    
    if cat_cols_for_imputation:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        train_data[cat_cols_for_imputation] = cat_imputer.fit_transform(train_data[cat_cols_for_imputation])
        print(f"\nОставшиеся категориальные пропуски в {cat_cols_for_imputation} заполнены модой.")
    else:
        print("\nНет категориальных колонок для общей импутации модой.")
        cat_imputer = None # Инициализируем

    # Проверка пропусков после всех импутаций
    print("\nПроцент пропусков после всех этапов импутации:")
    nan_after_imputation = (train_data.isnull().mean() * 100).reset_index()
    nan_after_imputation.columns = ["column_name", "percentage"]
    print(nan_after_imputation[nan_after_imputation["percentage"] > 0])
    if nan_after_imputation["percentage"].sum() == 0:
        print("Все пропуски в train_data устранены.")
else:
    print("Пропуск обработки пропусков, так как train_data пуст.")
    num_imputer = None
    cat_imputer = None


Числовые пропуски в ['movie_year', 'budget', 'domestic', 'international', 'run_time', 'director_experience', 'writer_experience', 'producer_experience', 'composer_experience', 'cinematographer_experience', 'main_actor_1_experience', 'main_actor_2_experience', 'main_actor_3_experience', 'main_actor_4_experience', 'cast_popularity'] заполнены медианой.

Групповая импутация для категориальных признаков (модой по 'director')...
  Пропуски в 'cinematographer' заполнены модой по группам 'director'.
  Пропуски в 'composer' заполнены модой по группам 'director'.
  Пропуски в 'producer' заполнены модой по группам 'director'.
  Пропуски в 'writer' заполнены модой по группам 'director'.

Заполнение специфических пропусков значением 'Unknown'...
  Пропуски в 'genre_2' заполнены 'Unknown'.
  Пропуски в 'genre_3' заполнены 'Unknown'.
  Пропуски в 'genre_4' заполнены 'Unknown'.
  Пропуски в 'main_actor_4' заполнены 'Unknown'.

Оставшиеся категориальные пропуски в ['director', 'writer', 'producer', '

In [18]:
# Блок 7: Кодирование категориальных признаков

if not train_data.empty:
    # Анализ категориальных признаков перед кодированием
    cat_cols_original_types = train_data.select_dtypes(include=['object', 'category'])
    if not cat_cols_original_types.empty:
        cat_stats = pd.DataFrame({
            'Column': cat_cols_original_types.columns,
            'Unique_Count': [train_data[col].nunique() for col in cat_cols_original_types.columns]
        })
        cat_stats = cat_stats.sort_values(by='Unique_Count', ascending=False)
        print("\nСтатистика по категориальным столбцам (до кодирования):")
        print(cat_stats)
    else:
        print("\nНет категориальных столбцов для анализа перед кодированием.")

    # Определение столбцов для One-Hot Encoding и Target Encoding
    # Эти списки должны содержать имена столбцов, которые вы предполагаете кодировать этими методами.
    # Код ниже отфильтрует их, оставив только существующие и подходящие по типу.
    
    candidate_target_encoding_cols = ['main_actor_4', 'main_actor_3', 'writer', 'main_actor_2', 'producer', 'director', 'main_actor_1', 'cinematographer', 'composer', 'distributor']
    candidate_one_hot_cols = ['genre_2', 'genre_3', 'genre_4', 'genre_1', 'mpaa']

    # Отбираем только те столбцы, что реально есть в train_data и являются категориальными (object)
    # Важно: Target Encoding обычно применяется к столбцам с высокой кардинальностью, OHE - с низкой.
    columns_for_target_encoding = []
    for col in candidate_target_encoding_cols:
        if col in train_data.columns and train_data[col].dtype == 'object':
            columns_for_target_encoding.append(col)
        elif col in train_data.columns:
            print(f"  Предупреждение: Столбец '{col}' для Target Encoding не является object, его тип: {train_data[col].dtype}. Пропущен.")


    columns_for_one_hot = []
    for col in candidate_one_hot_cols:
        if col in train_data.columns and train_data[col].dtype == 'object':
            columns_for_one_hot.append(col)
        elif col in train_data.columns:
            print(f"  Предупреждение: Столбец '{col}' для One-Hot Encoding не является object, его тип: {train_data[col].dtype}. Пропущен.")


    print(f"\nСтолбцы, выбранные для Target Encoding: {columns_for_target_encoding}")
    print(f"Столбцы, выбранные для One-Hot Encoding: {columns_for_one_hot}")

    # 1. One-Hot Encoding
    if columns_for_one_hot:
        train_data_shape_before_ohe = train_data.shape
        train_data = pd.get_dummies(train_data, columns=columns_for_one_hot, dummy_na=False) # dummy_na=False стандартно
        print(f"\nВыполнено One-Hot Encoding для: {columns_for_one_hot}.")
        print(f"Размер train_data изменился с {train_data_shape_before_ohe} на {train_data.shape}")
    else:
        print("\nНет столбцов для One-Hot Encoding.")

    # 2. Target Encoding
    if columns_for_target_encoding:
        if 'worldwide' in train_data.columns and not train_data['worldwide'].isnull().any():
            target_encoder = ce.TargetEncoder(cols=columns_for_target_encoding, handle_missing='value', handle_unknown='value')
            # handle_missing='value' и handle_unknown='value' заменят пропуски и неизвестные категории средним значением по таргету.
            # Это может быть полезно, если в тестовых данных появятся новые категории.
            train_data[columns_for_target_encoding] = target_encoder.fit_transform(train_data[columns_for_target_encoding], train_data['worldwide'])
            print(f"\nВыполнено Target Encoding для: {columns_for_target_encoding}.")
        else:
            print("\nОшибка: Целевая переменная 'worldwide' отсутствует или содержит NaN. Target Encoding не выполнен.")
            target_encoder = None 
    else:
        print("\nНет столбцов для Target Encoding.")
        target_encoder = None 

    print("\nПервые 5 строк train_data после кодирования:")
    print(train_data.head())
    # print("Типы данных после кодирования:") # Может быть очень длинный вывод
    # print(train_data.info())
else:
    print("Пропуск кодирования признаков, так как train_data пуст.")
    target_encoder = None
    columns_for_target_encoding = [] # Инициализация на случай, если train_data пуст
    columns_for_one_hot = []       # Инициализация


Статистика по категориальным столбцам (до кодирования):
             Column  Unique_Count
8      main_actor_4          2136
7      main_actor_3          1898
1            writer          1863
6      main_actor_2          1564
2          producer          1385
0          director          1380
5      main_actor_1          1173
4   cinematographer           787
3          composer           713
14      distributor           165
11          genre_2            21
12          genre_3            21
13          genre_4            18
10          genre_1            15
9              mpaa             5

Столбцы, выбранные для Target Encoding: ['main_actor_4', 'main_actor_3', 'writer', 'main_actor_2', 'producer', 'director', 'main_actor_1', 'cinematographer', 'composer', 'distributor']
Столбцы, выбранные для One-Hot Encoding: ['genre_2', 'genre_3', 'genre_4', 'genre_1', 'mpaa']

Выполнено One-Hot Encoding для: ['genre_2', 'genre_3', 'genre_4', 'genre_1', 'mpaa'].
Размер train_data изменился с (3

In [19]:
if not train_data.empty and 'worldwide' in train_data.columns:
    # Отделение признаков (X) и целевой переменной (y)
    X = train_data.drop("worldwide", axis=1)
    y = train_data["worldwide"]
    # y = np.log1p(train_data["worldwide"]) # Если планируете логарифмировать цель

    print("\nПризнаки (X) и целевая переменная (y) разделены.")
    print("Размер X:", X.shape)
    print("Размер y:", y.shape)

    # Сохраним имена столбцов X для последующего использования (например, при сохранении артефактов)
    final_feature_columns = X.columns.tolist() # Это столбцы ПОСЛЕ всех кодирований

    # Масштабирование признаков
    scaler = StandardScaler()
    # scaler = MinMaxScaler() # Альтернативный вариант
    X_scaled = scaler.fit_transform(X)
    print("\nПризнаки X масштабированы с использованием StandardScaler.")

    # Разделение на обучающую и валидационную выборки (для оценки модели)
    X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    print(f"Размеры выборок для обучения/валидации: X_train: {X_train.shape}, X_val: {X_val.shape}")

else:
    print("Пропуск подготовки X, y и масштабирования: train_data пуст или отсутствует 'worldwide'.")
    X, y, X_scaled, X_train, X_val, y_train, y_val = [None]*7 # Заглушки
    final_feature_columns = []
    scaler = None


Признаки (X) и целевая переменная (y) разделены.
Размер X: (3998, 105)
Размер y: (3998,)

Признаки X масштабированы с использованием StandardScaler.
Размеры выборок для обучения/валидации: X_train: (3198, 105), X_val: (800, 105)


In [20]:
# Блок 9: Обучение модели XGBoost с RandomizedSearchCV

best_model = None 

if X_train is not None and y_train is not None:
    print("\nНачало обучения модели XGBoost с RandomizedSearchCV...")
    
    param_dist_xgb = {
        'n_estimators': [100, 300, 500, 700, 1000], # Добавлено 1000
        'max_depth':    [4, 6, 8, 10, 12],          # Добавлено 12
        'learning_rate':[0.01, 0.03, 0.05, 0.1],    # Добавлено 0.03
        'subsample':    [0.6, 0.7, 0.8, 0.9, 1.0],  # Расширен диапазон
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],# Расширен диапазон
        'reg_alpha':    [0, 0.01, 0.1, 0.5, 1],     # Добавлен 0.01
        'reg_lambda':   [0.5, 1, 2, 5]              # Изменен диапазон, убран 0
    }

    xgb_base_model = XGBRegressor(
        random_state=42,
        tree_method='hist', 
        n_jobs=-1 
    )

    random_search_xgb = RandomizedSearchCV(
        estimator=xgb_base_model,
        param_distributions=param_dist_xgb,
        n_iter=50, # Увеличено с 25 до 50 (или даже 100, если есть время)
        scoring='neg_mean_squared_error', 
        cv=3, 
        verbose=1,
        n_jobs=1, 
        random_state=42
    )

    print(f"Запускается RandomizedSearchCV с {random_search_xgb.n_iter} итерациями и {random_search_xgb.cv} фолдами кросс-валидации...")
    random_search_xgb.fit(X_train, y_train)

    best_model = random_search_xgb.best_estimator_
    print("\nЛучшие параметры для XGBoost:", random_search_xgb.best_params_)
    
    y_pred_val = best_model.predict(X_val)
    r2_val = r2_score(y_val, y_pred_val)
    mse_val = mean_squared_error(y_val, y_pred_val)
    mae_val = mean_absolute_error(y_val, y_pred_val)

    print(f"\nОценка лучшей модели XGBoost на валидационной выборке:")
    print(f"  R²: {r2_val:.4f}")
    print(f"  MSE: {mse_val:.2f}")
    print(f"  MAE: {mae_val:.2f}")
else:
    print("Пропуск обучения модели: X_train или y_train не определены.")


Начало обучения модели XGBoost с RandomizedSearchCV...
Запускается RandomizedSearchCV с 50 итерациями и 3 фолдами кросс-валидации...
Fitting 3 folds for each of 50 candidates, totalling 150 fits

Лучшие параметры для XGBoost: {'subsample': 0.8, 'reg_lambda': 5, 'reg_alpha': 1, 'n_estimators': 700, 'max_depth': 8, 'learning_rate': 0.03, 'colsample_bytree': 0.8}

Оценка лучшей модели XGBoost на валидационной выборке:
  R²: 0.9988
  MSE: 44882512123242.16
  MAE: 3057717.82


In [21]:
# Проверяем, что все необходимые компоненты существуют перед сохранением
if best_model and num_imputer and cat_imputer and target_encoder and scaler and final_feature_columns:
    print("\n--- НАЧАЛО: СОХРАНЕНИЕ АРТЕФАКТОВ ---")

    # 1. Сохранение обученной модели
    joblib.dump(best_model, "movie_box_office_model.joblib")
    print("  Модель 'movie_box_office_model.joblib' сохранена.")

    # 2. Сохранение численного импьютера
    joblib.dump(num_imputer, "numerical_imputer.joblib")
    print("  Численный импьютер 'numerical_imputer.joblib' сохранен.")

    # 3. Сохранение категориального импьютера
    joblib.dump(cat_imputer, "categorical_imputer.joblib")
    print("  Категориальный импьютер 'categorical_imputer.joblib' сохранен.")

    # 4. Сохранение Target Encoder
    joblib.dump(target_encoder, "target_encoder.joblib")
    print("  Target Encoder 'target_encoder.joblib' сохранен.")

    # 5. Сохранение Scaler
    joblib.dump(scaler, "scaler.joblib")
    print("  Scaler 'scaler.joblib' сохранен.")

    # 6. Сохранение информации о столбцах и картах преобразований
    
    # Списки столбцов, определенные ранее (убедитесь, что они актуальны)
    # columns_to_drop_initial уже определен
    # num_cols_for_imputation уже определен
    # cat_cols_for_imputation уже определен
    # columns_for_target_encoding уже определен
    # columns_for_one_hot уже определен
    # final_feature_columns уже определен (это X.columns.tolist() после всех кодировок)
    # cols_to_fill_by_director_mode уже определен
    # cols_to_fill_unknown_specific уже определен

    column_info = {
        "columns_to_drop_on_load": columns_to_drop_initial, # Столбцы, которые удалялись в самом начале
        "num_cols_imputed_on_train": num_cols_for_imputation, # Список числовых столбцов, к которым применялся num_imputer
        "cat_cols_imputed_on_train": cat_cols_for_imputation, # Список категориальных столбцов, к которым применялся cat_imputer
        "target_encoded_cols": columns_for_target_encoding, # Список столбцов для Target Encoding
        "one_hot_encoded_cols": columns_for_one_hot, # Список столбцов для One-Hot Encoding
        "final_model_features": final_feature_columns, # Итоговый список признаков для модели (порядок важен!)
        "experience_maps": experience_maps_to_save, # Карты для признаков "опыта"
        "grouped_imputation_maps": grouped_imputation_maps_to_save, # Карты для групповой импутации
        "group_impute_director_dependent_cols": cols_to_fill_by_director_mode, # Столбцы, импьютируемые по 'director'
        "fill_unknown_cols": cols_to_fill_unknown_specific, # Столбцы, заполняемые 'Unknown'
        "target_variable_name": "worldwide" # Имя целевой переменной
    }

    # Очистка np.nan для JSON сериализации в картах
    if "experience_maps" in column_info:
        column_info["experience_maps"] = {
            map_name: {str(k): (None if pd.isna(v) else v) for k, v in map_dict.items()}
            for map_name, map_dict in column_info["experience_maps"].items()
        }
    if "grouped_imputation_maps" in column_info:
         column_info["grouped_imputation_maps"] = {
            map_name: {str(k): (None if pd.isna(v) else v) for k, v in map_dict.items()}
            for map_name, map_dict in column_info["grouped_imputation_maps"].items()
        }

    with open("column_info.json", "w", encoding='utf-8') as f:
        json.dump(column_info, f, indent=4, ensure_ascii=False)
    print("  Информация о столбцах и картах 'column_info.json' сохранена.")

    print("--- АРТЕФАКТЫ УСПЕШНО СОХРАНЕНЫ ---")
else:
    print("\n--- СОХРАНЕНИЕ АРТЕФАКТОВ ПРОПУЩЕНО ---")
    print("  Одна или несколько необходимых компонент (модель, импьютеры, кодеры, скейлер, списки столбцов) не были созданы.")


--- НАЧАЛО: СОХРАНЕНИЕ АРТЕФАКТОВ ---
  Модель 'movie_box_office_model.joblib' сохранена.
  Численный импьютер 'numerical_imputer.joblib' сохранен.
  Категориальный импьютер 'categorical_imputer.joblib' сохранен.
  Target Encoder 'target_encoder.joblib' сохранен.
  Scaler 'scaler.joblib' сохранен.
  Информация о столбцах и картах 'column_info.json' сохранена.
--- АРТЕФАКТЫ УСПЕШНО СОХРАНЕНЫ ---


In [22]:
# Загрузка test.csv, созданного ранее
try:
    final_test_df = pd.read_csv("test.csv")
    print(f"\nЗагружен test.csv для финальной оценки (размер: {final_test_df.shape})")

    # Сохраняем истинные значения 'worldwide' и названия фильмов для вывода
    y_true_final_test = final_test_df['worldwide'].copy()
    movie_titles_final_test = final_test_df['movie_title'].copy()
    
    # --- НАЧАЛО ПОВТОРЕНИЯ ПРЕДОБРАБОТКИ для test.csv ---
    # Важно: здесь нужно ПОВТОРИТЬ все шаги предобработки, как для train_data,
    # но используя .transform() для обученных объектов и сохраненные карты.

    df_test_processed = final_test_df.drop(columns=columns_to_drop_initial, errors='ignore')

    # 1. Feature Engineering (опыт) - используем сохраненные experience_maps_to_save
    for col_personnel in personnel_cols_for_experience: # Из Блока 5
        map_key = f"{col_personnel}_experience_map"
        if col_personnel in df_test_processed.columns and map_key in experience_maps_to_save:
            current_map = experience_maps_to_save[map_key]
            df_test_processed[f"{col_personnel}_experience"] = df_test_processed[col_personnel].map(current_map).fillna(0)
    
    actor_exp_cols_test = []
    for i_actor in range(1, 5):
        col_actor = f"main_actor_{i_actor}"
        map_key_actor = f"{col_actor}_experience_map"
        if col_actor in df_test_processed.columns and map_key_actor in experience_maps_to_save:
            current_map_actor = experience_maps_to_save[map_key_actor]
            exp_col_name_actor = f"{col_actor}_experience"
            df_test_processed[exp_col_name_actor] = df_test_processed[col_actor].map(current_map_actor).fillna(0)
            actor_exp_cols_test.append(exp_col_name_actor)
    if actor_exp_cols_test:
        df_test_processed["cast_popularity"] = df_test_processed[actor_exp_cols_test].sum(axis=1)
    else:
        df_test_processed["cast_popularity"] = 0

    # 2. run_time
    if 'run_time' in df_test_processed.columns:
        df_test_processed['run_time'] = df_test_processed['run_time'].apply(convert_runtime_to_minutes)

    # 3. Numerical Imputation - используем обученный num_imputer
    if num_imputer and any(col in df_test_processed.columns for col in num_cols_for_imputation):
        cols_to_impute_test_num = [col for col in num_cols_for_imputation if col in df_test_processed.columns]
        if cols_to_impute_test_num:
             df_test_processed[cols_to_impute_test_num] = num_imputer.transform(df_test_processed[cols_to_impute_test_num])


    # 4. Grouped Imputation - используем сохраненные grouped_imputation_maps_to_save
    if 'director' in df_test_processed.columns:
        for col_group_impute in cols_to_fill_by_director_mode: # Из Блока 6
            map_key_group = f"{col_group_impute}_director_mode_map"
            if col_group_impute in df_test_processed.columns and map_key_group in grouped_imputation_maps_to_save:
                current_map_group = grouped_imputation_maps_to_save[map_key_group]
                # ВАЖНО: ключи в current_map_group могут быть числами, если director был числовым.
                # Преобразуем их в строки, если map был сохранен с str(k)
                # Если map был сохранен с оригинальными типами, то str() не нужен
                # current_map_group_fixed_keys = {str(k): v for k,v in current_map_group.items()}
                # mapped_modes_test = df_test_processed['director'].map(current_map_group_fixed_keys)

                # Если ключи в JSON были сохранены как строки (из-за str(k) в блоке сохранения)
                # а значения в df_test_processed['director'] - числа, то map не сработает.
                # Лучше при загрузке JSON обратно конвертировать ключи в числа, если нужно,
                # или убедиться, что тип ключа в map совпадает с типом значений в столбце.
                # Для простоты здесь предполагаем, что типы совпадают или map() обработает.
                mapped_modes_test = df_test_processed['director'].map(current_map_group)
                df_test_processed[col_group_impute] = df_test_processed[col_group_impute].fillna(mapped_modes_test)


    # 5. Fill 'Unknown'
    for col_fill_unknown in cols_to_fill_unknown_specific: # Из Блока 6
        if col_fill_unknown in df_test_processed.columns:
            df_test_processed[col_fill_unknown] = df_test_processed[col_fill_unknown].fillna('Unknown')
    
    # 6. Categorical Imputation - используем обученный cat_imputer
    if cat_imputer and any(col in df_test_processed.columns for col in cat_cols_for_imputation):
        cols_to_impute_test_cat = [col for col in cat_cols_for_imputation if col in df_test_processed.columns and df_test_processed[col].dtype == 'object']
        if cols_to_impute_test_cat:
            df_test_processed[cols_to_impute_test_cat] = cat_imputer.transform(df_test_processed[cols_to_impute_test_cat])

    # 7. One-Hot Encoding - применяем get_dummies
    if columns_for_one_hot: # Из Блока 7
        cols_ohe_test = [col for col in columns_for_one_hot if col in df_test_processed.columns]
        if cols_ohe_test:
            df_test_processed = pd.get_dummies(df_test_processed, columns=cols_ohe_test, dummy_na=False)

    # 8. Target Encoding - используем обученный target_encoder
    if target_encoder and columns_for_target_encoding: # Из Блока 7
        cols_te_test = [col for col in columns_for_target_encoding if col in df_test_processed.columns]
        if cols_te_test:
            df_test_processed[cols_te_test] = target_encoder.transform(df_test_processed[cols_te_test])
            
    # 9. Reindex - ВАЖНО: привести столбцы тестового набора в соответствие с обучающим
    # `final_feature_columns` - это X.columns.tolist() из обучающего набора ПОСЛЕ всех кодировок
    df_test_processed = df_test_processed.reindex(columns=final_feature_columns, fill_value=0)
    
    # Проверка на NaN перед масштабированием в тестовых данных
    if df_test_processed.isnull().any().any():
        print("\nПРЕДУПРЕЖДЕНИЕ: Обнаружены NaN в тестовых данных ПОСЛЕ предобработки и ПЕРЕД масштабированием!")
        print(df_test_processed.isnull().sum()[df_test_processed.isnull().sum() > 0])
        # Можно добавить принудительное заполнение нулями или другой стратегией, если это допустимо
        # df_test_processed.fillna(0, inplace=True) 
        # print("NaN принудительно заменены на 0.")


    # 10. Scaling - используем обученный scaler
    if scaler:
        X_final_test_scaled = scaler.transform(df_test_processed)
    else:
        X_final_test_scaled = df_test_processed.values # Если скейлер не был создан (маловероятно)

    # --- КОНЕЦ ПОВТОРЕНИЯ ПРЕДОБРАБОТКИ ---

    # Предсказание на обработанных тестовых данных
    if best_model and X_final_test_scaled is not None:
        y_pred_final_test = best_model.predict(X_final_test_scaled)

        # Оценка
        r2_final = r2_score(y_true_final_test, y_pred_final_test)
        mse_final = mean_squared_error(y_true_final_test, y_pred_final_test)
        mae_final = mean_absolute_error(y_true_final_test, y_pred_final_test)

        print(f"\nФинальная оценка модели на test.csv (загруженном из файла):")
        print(f"  R²: {r2_final:.4f}")
        print(f"  MSE: {mse_final:.2f}")
        print(f"  MAE: {mae_final:.2f}")

        # Вывод нескольких примеров предсказаний
        results_df = pd.DataFrame({
            'Movie Title': movie_titles_final_test,
            'Actual Worldwide Gross': y_true_final_test,
            'Predicted Worldwide Gross': y_pred_final_test
        })
        print("\nПримеры предсказаний на test.csv:")
        print(results_df.head(10))

except FileNotFoundError:
    print("\nФайл test.csv не найден для финальной оценки в этом ноутбуке.")
except Exception as e:
    print(f"\nПроизошла ошибка при финальной оценке на test.csv: {e}")
    import traceback
    traceback.print_exc()


Загружен test.csv для финальной оценки (размер: (1714, 24))

Финальная оценка модели на test.csv (загруженном из файла):
  R²: 0.9956
  MSE: 185828245543836.97
  MAE: 7655874.56

Примеры предсказаний на test.csv:
                            Movie Title  Actual Worldwide Gross  \
0                 Rise of the Guardians             306941670.0   
1              The Fast and the Furious             207284863.0   
2     Dickie Roberts: Former Child Star              23769505.0   
3                                 Dumbo             353284621.0   
4                          The Predator             160542134.0   
5                   Along Came a Spider             105178561.0   
6                        Last Christmas             121550750.0   
7  The Visual Bible: The Gospel of John               4078741.0   
8                         Lilo & Stitch             273144151.0   
9                     The Jungle Book 2             186303759.0   

   Predicted Worldwide Gross  
0                